<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/D-Wave_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

**Possibly need to restart runtime here.**

In [ ]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

In [ ]:
#!curl  -H "X-Auth-Token: YourAPIkey" -X DELETE https://cloud.dwavesys.com/sapi/problems/7211d196-cfc6-4630-84d0-f42ae5d55f0f

In [ ]:
#https://www.youtube.com/watch?v=bErs0dxC1aY&list=PLPvKnT7dgEsuhec_yW9oxcZ6gg9SfRFFV
#https://docs.ocean.dwavesys.com/en/latest/overview/install.html#installoceansoftware
#!python -m venv ocean
#!\Scripts\activate

!pip install dwave-ocean-sdk

In [ ]:
!pip install requests --upgrade

In [ ]:
!dwave setup #API endpoint URL: https://cloud.dwavesys.com/sapi/

# 1. Load Data

In [12]:
!pwd

/content


In [13]:
#next run the model.py in the folder introScenario
!pwd
#%cd ../intermediateScenario/
#%cd ./Summer2021/intermediateScenario
%cd ./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [14]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [15]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [122]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [17]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [18]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model


In [19]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [20]:
exec(open("./model.py").read())

* building wado model
model building done in 0.15884113311767578 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

In [21]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [22]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [23]:
model

docplex.mp.Model['docplex_model1']

In [105]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [106]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


# 3. Solve the problem using D-wave

In [139]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization.translators import from_docplex_mp

from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.algorithms import CplexOptimizer
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs
cplex = CplexOptimizer()#for binary and integer variables
cobyla = CobylaOptimizer()#for continues variables


from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [141]:
lc_name='_L_EXPR_19'
lin_c = mod.get_linear_constraint(lc_name)
print('lin_c:',lc_name,lin_c.linear.to_dict(use_name=True), lin_c.sense, lin_c.rhs)
mod.remove_linear_constraint(lc_name)
mod.linear_constraint(lin_c.linear.to_dict(use_name=True), sense='==', rhs=lin_c.rhs, name=lc_name)
print(mod.export_as_lp_string())


lin_c: _L_EXPR_19 {'x2': 1.0, 'x3': 1.0} ConstraintSense.LE 100
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120
 _L_EXPR_19: x2 + x3 = 100

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [142]:
constant =mod.objective.constant
linear=s_df['Cost'].values
quadratic=np.kron(linear.reshape(1,-1),linear.reshape(-1,1))
linear=mod.objective.linear.to_array()
mod.minimize(constant=constant, linear=linear, quadratic=quadratic)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5 + [ 18 x0^2
      + 24 x0*x1 + 12 x0*x2 + 36 x0*x3 + 24 x0*x4 + 12 x0*x5 + 8 x1^2 + 8 x1*x2
      + 24 x1*x3 + 16 x1*x4 + 8 x1*x5 + 2 x2^2 + 12 x2*x3 + 8 x2*x4 + 4 x2*x5
      + 18 x3^2 + 24 x3*x4 + 12 x3*x5 + 8 x4^2 + 8 x4*x5 + 2 x5^2 ]/2
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120
 _L_EXPR_19: x2 + x3 = 100

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1


In [148]:

print('\n',model.solve())

solution=cplex.solve(mod)
print(solution)



 solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



KeyboardInterrupt: ignored

In [144]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -496.0, 3: -464.0, 4: -480.0, 5: -496.0}
linear array:			 [-464. -480. -496. -464. -480. -496.    0.    0.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-496.0
  (0, 3)	-464.0
  (0, 4)	-480.0
  (0, 5)	-496.0 

quadratic dict w/ index:	 {(0, 0): 9, (0, 1): 12, (1, 1): 4, (0, 2): 6, (1, 2): 4, (2, 2): 1, (0, 3): 18, (1, 3): 12, (2, 3): 6, (3, 3): 9, (0, 4): 12, (1, 4): 8, (2, 4): 4, (3, 4): 12, (4, 4): 4, (0, 5): 6, (1, 5): 4, (2, 5): 2, (3, 5): 6, (4, 5): 4, (5, 5): 1}
quadratic dict w/ name:		 {('x0', 'x0'): 9, ('x0', 'x1'): 12, ('x1', 'x1'): 4, ('x0', 'x2'): 6, ('x1', 'x2'): 4, ('x2', 'x2'): 1, ('x0', 'x3'): 18, ('x1', 'x3'): 12, ('x2', 'x3'): 6, ('x3', 'x3'): 9, ('x0', 'x4'): 12, ('x1', 'x4'): 8, ('x2', 'x4'): 4, ('x3', 'x4'): 12, ('x4', 'x4'): 4, ('x0', 'x5'): 6, ('x1', 'x5'): 4, ('x2', 'x5'): 2, ('x3', 'x5'): 6, ('x4', 'x5'): 4, ('x5', 'x5'): 1}
symmetric quadratic dict w/

In [145]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)

In [146]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')
print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

constant:			 6.2532e+46
linear array as sparse matrix:
   (0, 0)	-4.608e+44
  (0, 1)	-9.216e+44
  (0, 2)	-1.8432e+45
  (0, 3)	-3.6864e+45
  (0, 4)	-7.3728e+45
  (0, 5)	-1.47456e+46
  (0, 6)	-2.94912e+46
  (0, 7)	-5.89824e+46
  (0, 8)	-1.179648e+47
  (0, 9)	-2.359296e+47
  (0, 10)	-4.718592e+47
  (0, 11)	-9.437184e+47
  (0, 12)	-1.8874368e+48
  (0, 13)	-3.7748736e+48
  (0, 14)	-7.5497472e+48
  (0, 15)	-1.50994944e+49
  (0, 16)	-3.01989888e+49
  (0, 17)	-6.03979776e+49
  (0, 18)	-1.207959552e+50
  (0, 19)	-2.415919104e+50
  (0, 20)	-4.831838208e+50
  (0, 21)	-9.663676416e+50
  (0, 22)	-1.9327352832e+51
  (0, 23)	-3.8654705664e+51
  (0, 24)	-7.7309411328e+51
  :	:
  (0, 855)	-1.2960000000000001e+45
  (0, 856)	-8.64e+43
  (0, 857)	-1.728e+44
  (0, 858)	-3.456e+44
  (0, 859)	-6.912e+44
  (0, 860)	-1.2960000000000001e+45
  (0, 861)	-2.88e+44
  (0, 862)	-5.76e+44
  (0, 863)	-1.152e+45
  (0, 864)	-2.304e+45
  (0, 865)	-4.608e+45
  (0, 866)	-9.216e+45
  (0, 867)	-1.0656000000000002e+46
  (0, 86

In [135]:
from qiskit.optimization.converters import InequalityToEquality
from qiskit.optimization.converters import IntegerToBinary
from qiskit.optimization.converters import LinearEqualityToPenalty

ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(mod)
#print(qp_eq.export_as_lp_string())

int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
#print(qp_eq_bin.export_as_lp_string())

lineq2penalty = LinearEqualityToPenalty()
mod_Qubo2 = lineq2penalty.convert(qp_eq_bin)
#print(qubo.export_as_lp_string())

len(mod_Qubo2._variables_index)

880

In [ ]:
qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [147]:
import dimod
import hybrid

import neal #sampler = neal.SimulatedAnnealingSampler()

from dwave.system import LeapHybridSampler
from dwave.system import DWaveSampler, EmbeddingComposite


# Extract qubo data

qubo=mod_Qubo

Q = qubo.objective.quadratic.to_dict(use_name=True)
g = qubo.objective.linear.to_dict(use_name=True)
c = qubo.objective.constant

# Build BQM
bqm = dimod.BQM(g, Q, c, dimod.BINARY)

# Solve
sampler = LeapHybridSampler()
bqm.result = sampler.sample(bqm)

# Extract solution
result_dict = bqm.result.first.sample

print(bqm.result)

SolverFailureError: ignored